# Event Extraction Agent 테스트 (Production Level)

이벤트 추출 에이전트 (Level 1) 테스트 노트북

## 주요 테스트 케이스
1. 기본 이벤트 추출
2. 캐릭터 연동 테스트 (Production Level 캐릭터 데이터와 함께)
3. Re-extraction (충돌 해결)

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 키가 크고 검은 머리카락을 가진 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다. 그녀는 하얀 치료사 로브를 입고 있었다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다. 그의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. 검은 갑옷을 입은 그는 예전보다 더 날카로워 보였다.
서진은 용감하게 앞으로 나섰다. 긴장감이 최고조에 달했다. "왜 마을을 배신했어, 민호?"

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!" 그녀의 목소리에는 급박함이 묻어났다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_events": [], "extracted_characters": [],
            "consistency_report": {}, "retry_count": 0, 
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

def pretty_print(title, data):
    print(f"\n{'='*70}")
    print(f"📋 {title}")
    print(f"{'='*70}")
    print(json.dumps(data, ensure_ascii=False, indent=2))

## 1. 기본 이벤트 추출 테스트

In [3]:
from app.agents.extraction.event import event_extraction_node

async def test_event_extraction():
    print("📅 Event Extraction 테스트...")
    state = create_base_state()
    return await event_extraction_node(state)

result = run_async(test_event_extraction())
pretty_print("이벤트 추출 결과", result)

📅 Event Extraction 테스트...

📋 이벤트 추출 결과
{
  "extracted_events": [
    {
      "event_id": "E001",
      "description": "서진이 어두운 숲 속에서 검을 쥐고 있었다.",
      "participants": [
        "서진"
      ],
      "event_type": "action",
      "importance": 5
    },
    {
      "event_id": "E002",
      "description": "서진은 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.",
      "participants": [
        "서진"
      ],
      "event_type": "revelation",
      "importance": 7
    },
    {
      "event_id": "E003",
      "description": "하나가 서진에게 그를 만날 수 있을지 물었다.",
      "participants": [
        "하나",
        "서진"
      ],
      "event_type": "dialogue",
      "importance": 6
    },
    {
      "event_id": "E004",
      "description": "하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다.",
      "participants": [
        "하나"
      ],
      "event_type": "revelation",
      "importance": 6
    },
    {
      "event_id": "E005",
      "description": "이민호라는 그림자가 나무 사이에서 나타났다.",
      "participants": [
        "이민호"
      ],
      "event_type": 

## 2. 이벤트 분석

In [4]:
events = result.get("extracted_events", [])

print("\n" + "="*70)
print("📊 이벤트 분석")
print("="*70)

for event in events:
    print(f"\n📌 [{event.get('event_id')}] {event.get('description')}")
    print(f"   Type: {event.get('event_type')}")
    print(f"   Participants: {event.get('participants', [])}")
    print(f"   Importance: {event.get('importance', 'N/A')}/10")

# 통계
print(f"\n📈 총 이벤트 수: {len(events)}")
event_types = {}
for e in events:
    t = e.get('event_type', 'unknown')
    event_types[t] = event_types.get(t, 0) + 1
print(f"   유형별: {event_types}")


📊 이벤트 분석

📌 [E001] 서진이 어두운 숲 속에서 검을 쥐고 있었다.
   Type: action
   Participants: ['서진']
   Importance: 5/10

📌 [E002] 서진은 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
   Type: revelation
   Participants: ['서진']
   Importance: 7/10

📌 [E003] 하나가 서진에게 그를 만날 수 있을지 물었다.
   Type: dialogue
   Participants: ['하나', '서진']
   Importance: 6/10

📌 [E004] 하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다.
   Type: revelation
   Participants: ['하나']
   Importance: 6/10

📌 [E005] 이민호라는 그림자가 나무 사이에서 나타났다.
   Type: action
   Participants: ['이민호']
   Importance: 8/10

📌 [E006] 이민호는 한때 서진의 가장 친한 친구였지만, 지금은 그들의 마을을 배신한 적이 되어버렸다.
   Type: revelation
   Participants: ['이민호', '서진']
   Importance: 9/10

📌 [E007] 이민호의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.
   Type: revelation
   Participants: ['이민호']
   Importance: 7/10

📌 [E008] 이민호가 서진에게 오랜만이라고 차갑게 말했다.
   Type: dialogue
   Participants: ['이민호', '서진']
   Importance: 8/10

📌 [E009] 이민호는 예전보다 더 날카로워 보였다.
   Type: revelation
   Participants: ['이민호']
   Importance: 7/10

📌 [E010] 서진이 용감하게 앞으로 나섰고, 긴장감이 최고조에 달했다.

## 3. 캐릭터-이벤트 연결 분석 (Production Level 캐릭터 데이터 활용)

In [5]:
# Production Level 캐릭터 데이터 (character.py에서 추출된 형식)
production_characters = [
    {
        "name": "서진",
        "role": "protagonist",
        "visual": {
            "appearance": ["tall", "dark hair"],
            "attire": ["holding sword"]
        },
        "personality": {
            "core_traits": ["brave", "skilled swordsman"],
            "flaws": ["impulsive"]
        },
        "relationships": [
            {"target": "이민호", "type": "ENEMY", "history": "former_friend", "strength": 8}
        ],
        "current_mood": {"emotion": "tense", "intensity": 7}
    },
    {
        "name": "이민호",
        "role": "antagonist",
        "visual": {
            "appearance": ["cold eyes"],
            "attire": ["wearing black armor"]
        },
        "personality": {
            "core_traits": ["cold", "skilled", "betrayer"],
            "flaws": ["bitter", "vengeful"]
        },
        "relationships": [
            {"target": "서진", "type": "ENEMY", "history": "former_friend", "strength": 8}
        ],
        "current_mood": {"emotion": "angry", "intensity": 9}
    }
]

print("\n" + "="*70)
print("🔗 캐릭터-이벤트 연결 분석")
print("="*70)

# 캐릭터별 이벤트 참여도
char_events = {}
for event in events:
    for participant in event.get("participants", []):
        if participant not in char_events:
            char_events[participant] = []
        char_events[participant].append(event.get("event_id"))

for char in production_characters:
    name = char.get("name")
    mood = char.get("current_mood", {})
    print(f"\n🧑 {name} ({char.get('role')})")
    print(f"   Current Mood: {mood.get('emotion', 'N/A')} (intensity: {mood.get('intensity', 'N/A')})")
    print(f"   참여 이벤트: {char_events.get(name, [])}")
    
    # 관계 기반 분석
    for rel in char.get("relationships", []):
        target = rel.get("target")
        shared_events = set(char_events.get(name, [])) & set(char_events.get(target, []))
        if shared_events:
            print(f"   🔗 {target}와 공유 이벤트: {list(shared_events)} ({rel.get('type')}, was: {rel.get('history')})")


🔗 캐릭터-이벤트 연결 분석

🧑 서진 (protagonist)
   Current Mood: tense (intensity: 7)
   참여 이벤트: ['E001', 'E002', 'E003', 'E006', 'E008', 'E010', 'E011']
   🔗 이민호와 공유 이벤트: ['E008', 'E011', 'E006'] (ENEMY, was: former_friend)

🧑 이민호 (antagonist)
   Current Mood: angry (intensity: 9)
   참여 이벤트: ['E005', 'E006', 'E007', 'E008', 'E009', 'E011']
   🔗 서진와 공유 이벤트: ['E008', 'E011', 'E006'] (ENEMY, was: former_friend)


## 4. Re-extraction 테스트 (충돌 해결)

In [6]:
async def test_re_extraction():
    print("🔄 Event Re-extraction 테스트...")
    state = create_base_state()
    state["retry_count"] = 1
    
    # 이전 추출 결과
    state["extracted_events"] = [
        {
            "event_id": "E001", 
            "description": "서진과 하나가 숲에서 대기", 
            "participants": ["서진", "하나"],
            "event_type": "action",
            "importance": 3
        },
        {
            "event_id": "E002", 
            "description": "이민호 등장", 
            "participants": ["이민호"],
            "event_type": "action",
            "importance": 7
        },
        {
            "event_id": "E003", 
            "description": "서진과 이민호의 대화", 
            "participants": ["서진", "이민호"],
            "event_type": "action",  # 잘못됨 - dialogue여야 함
            "importance": 5  # 잘못됨 - 중요도 더 높아야 함
        }
    ]
    
    # 충돌 정보
    state["consistency_report"] = {
        "conflicts": [
            {
                "type": "EVENT_TYPE_CONFLICT",
                "description": "E003은 대화 장면이므로 event_type이 'dialogue'여야 함",
                "severity": "MEDIUM",
                "affected_elements": ["E003"]
            },
            {
                "type": "IMPORTANCE_CONFLICT", 
                "description": "E003은 스토리의 핵심 갈등을 드러내는 장면이므로 importance가 8 이상이어야 함",
                "severity": "LOW",
                "affected_elements": ["E003"]
            }
        ]
    }
    
    return await event_extraction_node(state)

re_result = run_async(test_re_extraction())
pretty_print("Re-extraction 결과", re_result)

🔄 Event Re-extraction 테스트...
[EVENT] Re-extracting with 2 conflicts as feedback

📋 Re-extraction 결과
{
  "extracted_events": [
    {
      "event_id": "E001",
      "description": "서진이 숲에서 검을 들고 있음",
      "participants": [
        "서진"
      ],
      "event_type": "action",
      "importance": 5
    },
    {
      "event_id": "E002",
      "description": "하나가 서진 옆에 서서 질문함",
      "participants": [
        "하나",
        "서진"
      ],
      "event_type": "dialogue",
      "importance": 5
    },
    {
      "event_id": "E003",
      "description": "이민호가 나타남",
      "participants": [
        "이민호"
      ],
      "event_type": "action",
      "importance": 8
    },
    {
      "event_id": "E004",
      "description": "서진과 이민호의 대화",
      "participants": [
        "서진",
        "이민호"
      ],
      "event_type": "dialogue",
      "importance": 8
    },
    {
      "event_id": "E005",
      "description": "박서연이 경비병이 오고 있다고 알림",
      "participants": [
        "박서연"
      ],
      "event_type"

In [7]:
# Re-extraction 분석
print("\n" + "="*70)
print("📊 Re-extraction 분석")
print("="*70)

re_events = re_result.get("extracted_events", [])

for event in re_events:
    event_id = event.get('event_id')
    if event_id == 'E003' or '대화' in event.get('description', ''):
        print(f"\n📌 [{event_id}] {event.get('description')}")
        print(f"   Type: {event.get('event_type')} {'✅' if event.get('event_type') == 'dialogue' else '❌'}")
        print(f"   Importance: {event.get('importance')}/10 {'✅' if event.get('importance', 0) >= 8 else '❌'}")


📊 Re-extraction 분석

📌 [E003] 이민호가 나타남
   Type: action ❌
   Importance: 8/10 ✅

📌 [E004] 서진과 이민호의 대화
   Type: dialogue ✅
   Importance: 8/10 ✅


## 5. 타임라인 시각화

In [8]:
print("\n" + "="*70)
print("📅 이벤트 타임라인")
print("="*70)

# 이벤트를 순서대로 표시
for i, event in enumerate(events, 1):
    importance = event.get('importance', 5)
    bar = '█' * importance + '░' * (10 - importance)
    event_type_emoji = {
        'action': '⚔️',
        'dialogue': '💬',
        'revelation': '💡',
        'flashback': '⏪'
    }.get(event.get('event_type', ''), '📌')
    
    print(f"\n{i}. {event_type_emoji} [{event.get('event_id')}] {event.get('description')}")
    print(f"   Importance: [{bar}] {importance}/10")
    print(f"   Participants: {', '.join(event.get('participants', []))}")


📅 이벤트 타임라인

1. ⚔️ [E001] 서진이 어두운 숲 속에서 검을 쥐고 있었다.
   Importance: [█████░░░░░] 5/10
   Participants: 서진

2. 💡 [E002] 서진은 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
   Importance: [███████░░░] 7/10
   Participants: 서진

3. 💬 [E003] 하나가 서진에게 그를 만날 수 있을지 물었다.
   Importance: [██████░░░░] 6/10
   Participants: 하나, 서진

4. 💡 [E004] 하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다.
   Importance: [██████░░░░] 6/10
   Participants: 하나

5. ⚔️ [E005] 이민호라는 그림자가 나무 사이에서 나타났다.
   Importance: [████████░░] 8/10
   Participants: 이민호

6. 💡 [E006] 이민호는 한때 서진의 가장 친한 친구였지만, 지금은 그들의 마을을 배신한 적이 되어버렸다.
   Importance: [█████████░] 9/10
   Participants: 이민호, 서진

7. 💡 [E007] 이민호의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.
   Importance: [███████░░░] 7/10
   Participants: 이민호

8. 💬 [E008] 이민호가 서진에게 오랜만이라고 차갑게 말했다.
   Importance: [████████░░] 8/10
   Participants: 이민호, 서진

9. 💡 [E009] 이민호는 예전보다 더 날카로워 보였다.
   Importance: [███████░░░] 7/10
   Participants: 이민호

10. ⚔️ [E010] 서진이 용감하게 앞으로 나섰고, 긴장감이 최고조에 달했다.
   Importance: [████████░░] 8/10
   Participants: 서진

11